In [1]:
# !pip install pandas

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(), override=True)

In [3]:
import pandas as pd
df = pd.read_csv('Data.csv', sep=';')

In [4]:
df.head()

,Product,Review
0,"MacBook Pro 14, M2",I ordered it in MediaMart in Poland. Very expe...
1,"VW Passat, B8",I bought it used but I like it.
2,PS5,I loved it. Wish it have more games in Ukraini...


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [6]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [7]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [8]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [9]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [10]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [11]:
review = df.Review[2]
overall_chain(review)



> Entering new  chain...

> Finished chain.


{'Review': 'I loved it. Wish it have more games in Ukrainian language.',
 'English_Review': 'Я любив це. Хотілося б, щоб було більше ігор українською мовою.\n\nI loved it. I wish there were more games in Ukrainian language.',
 'summary': 'The reviewer enjoyed the game and wished for more games in Ukrainian language.',
 'followup_message': "Thank you for your positive review! We're thrilled to hear that you enjoyed the game. We appreciate your feedback regarding more games in Ukrainian language and will definitely take your suggestion into consideration for future projects."}